In [1]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as utils
import torch.nn.functional as F

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.utils import save_image

from torchvision.transforms import Compose, CenterCrop, Normalize, ToTensor
from pytorch_wavelets import DWT1DForward, DWT1DInverse  # or simply DWT1D, IDWT1D

from glob import glob

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [3]:
batch_size = 64
test_size = 64
Noise = "-4_db"

def Normalization(data):
    
    data_mean = data.mean()
    data_std = data.std()
    
    data = data - data_mean
    data = data / data_std
    
    return data

In [4]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
print(torch.cuda.is_available())

True


In [5]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, root):
        self.root = root  
        if not os.path.exists(self.root):
            raise Exception("[!] {} not exists.".format(root))
        
        #sort file names
        self.input_paths = sorted(glob(os.path.join(self.root, '{}/*_train.npy'.format("IB_data/noise_data/"+Noise+"/train_data"))))
        self.label_paths = sorted(glob(os.path.join(self.root, '{}/*_lab.npy'.format("IB_data/noise_data/"+Noise+"/train_lab"))))
        self.name = os.path.basename(root)
        
        #print(self.input_paths)
        #print(self.label_paths)
        
        if len(self.input_paths) == 0 or len(self.label_paths) == 0:
            raise Exception("No signal/labels are found in {}".format(self.root))

    def __getitem__(self, index):
        
        Signal = np.load(self.input_paths[index])
        Signal = Normalization(Signal)
        
        Label = np.load(self.label_paths[index])
            
        return Signal, Label

    def __len__(self):
        return len(self.input_paths)
    
class Dataset_test(torch.utils.data.Dataset):
    def __init__(self, root):
        self.root = root
        
        if not os.path.exists(self.root):
            raise Exception("[!] {} not exists.".format(root))
        
        #sort file names
        self.input_paths = sorted(glob(os.path.join(self.root, '{}/*_test.npy'.format("IB_data/noise_data/"+Noise+"/test_data"))))
        self.label_paths = sorted(glob(os.path.join(self.root, '{}/*_lab.npy'.format("IB_data/noise_data/"+Noise+"/test_lab"))))
        self.name = os.path.basename(root)
        
        if len(self.input_paths) == 0 or len(self.label_paths) == 0:
            raise Exception("No sinagl/labels are found in {}".format(self.root))

    def __getitem__(self, index):
        
        Signal = np.load(self.input_paths[index])
        Signal = Normalization(Signal)
        
        Label = np.load(self.label_paths[index])
        
        return Signal, Label

    def __len__(self):
        return len(self.input_paths)

In [6]:
def loader(dataset, batch_size, num_workers=0, shuffle = False, drop_last=False):

    input_images = dataset
    input_loader = torch.utils.data.DataLoader(dataset=input_images, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, drop_last=drop_last)

    return input_loader

train_loader = loader(Dataset('../../'), batch_size= batch_size, shuffle = True, drop_last=True)
test_loader = loader(Dataset_test('../../'), batch_size= test_size, shuffle = True, drop_last=True)

In [7]:
input = 1
numf =12

class SConv_1D(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch, kernel, pad):
        super(SConv_1D, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv1d(in_ch, out_ch, kernel, padding=pad),
            nn.GroupNorm(6, out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        
        x = self.conv(x)
        return x


class WaveCNN_1D(nn.Module):
    def __init__(self):
        super(WaveCNN_1D, self).__init__()
        
        self.SConv1 = SConv_1D(input, numf, 5, 2)
        self.DWT1= DWT1DForward(J=3, wave='db12').cuda()
        self.dropout1 = nn.Dropout(p=0.1)

        
        self.SConv2 = SConv_1D(numf, numf*2, 5, 2)        
        self.pool2 = nn.MaxPool1d(4)
        self.DWT2= DWT1DForward(J=3, wave='db12').cuda() 
        self.dropout2 = nn.Dropout(p=0.1)
        
        
        self.SConv3 = SConv_1D(numf*2, numf*4, 5, 2)
        self.pool3 = nn.MaxPool1d(4)
        self.DWT3= DWT1DForward(J=3, wave='db12').cuda()       
        self.dropout3 = nn.Dropout(p=0.1)
        

        self.SConv4 = SConv_1D(numf*4, numf*8, 5, 2)
        self.pool4 = nn.MaxPool1d(4)
        self.DWT4= DWT1DForward(J=3, wave='db12').cuda()  
        self.dropout4 = nn.Dropout(p=0.1)
        
        
        self.SConv5 = SConv_1D(numf*8, numf*16, 5, 2) 
        self.pool5 = nn.MaxPool1d(2)
        self.DWT5= DWT1DForward(J=3, wave='db12').cuda()  
        self.dropout5 = nn.Dropout(p=0.1)
        
        self.SConv6 = SConv_1D(numf*16, numf*32, 5, 2)              
        
        self.avg_pool = nn.AdaptiveAvgPool1d((1))
        self.fc = nn.Linear(numf*32, 7)

        
    def forward(self, input):
        
        output = self.SConv1(input)
        DMT_yl1,DMT_yh1 = self.DWT1(output)
        output = torch.cat([DMT_yl1,DMT_yh1[2],DMT_yh1[1],DMT_yh1[0]], dim=2)
        output = self.dropout1(output)
        
        output = self.SConv2(output)
        output = self.pool2(output)
        DMT_yl2,DMT_yh2 = self.DWT2(output)
        output = torch.cat([DMT_yl2,DMT_yh2[2],DMT_yh2[1],DMT_yh2[0]], dim=2)
        output = self.dropout2(output)
        
        output = self.SConv3(output)
        output = self.pool3(output)        
        DMT_yl3,DMT_yh3 = self.DWT3(output)
        output = torch.cat([DMT_yl3,DMT_yh3[2],DMT_yh3[1],DMT_yh3[0]], dim=2)
        output = self.dropout3(output)
        
        output = self.SConv4(output)
        output = self.pool4(output)
        DMT_yl4,DMT_yh4 = self.DWT4(output)
        output = torch.cat([DMT_yl4,DMT_yh4[2],DMT_yh4[1],DMT_yh4[0]], dim=2)         
        output = self.dropout4(output)
        
        output = self.SConv5(output)
        output = self.pool5(output)
        DMT_yl5,DMT_yh5 = self.DWT5(output)
        output = torch.cat([DMT_yl5,DMT_yh5[2],DMT_yh5[1],DMT_yh5[0]], dim=2)    
        output = self.dropout5(output)
        
        output = self.SConv6(output)             
        #print(output.shape)    
        output = self.avg_pool(output)
        output = output.view(output.size(0), -1)
        output = self.fc(output)
        
        return output

In [8]:
Model = WaveCNN_1D().to(device)

print(Model)

print('# Model parameters:', sum(param.numel() for param in Model.parameters()))

WaveCNN_1D(
  (SConv1): SConv_1D(
    (conv): Sequential(
      (0): Conv1d(1, 12, kernel_size=(5,), stride=(1,), padding=(2,))
      (1): GroupNorm(6, 12, eps=1e-05, affine=True)
      (2): ReLU(inplace=True)
    )
  )
  (DWT1): DWT1DForward()
  (dropout1): Dropout(p=0.1, inplace=False)
  (SConv2): SConv_1D(
    (conv): Sequential(
      (0): Conv1d(12, 24, kernel_size=(5,), stride=(1,), padding=(2,))
      (1): GroupNorm(6, 24, eps=1e-05, affine=True)
      (2): ReLU(inplace=True)
    )
  )
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (DWT2): DWT1DForward()
  (dropout2): Dropout(p=0.1, inplace=False)
  (SConv3): SConv_1D(
    (conv): Sequential(
      (0): Conv1d(24, 48, kernel_size=(5,), stride=(1,), padding=(2,))
      (1): GroupNorm(6, 48, eps=1e-05, affine=True)
      (2): ReLU(inplace=True)
    )
  )
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (DWT3): DWT1DForward()
  (dropout3): Dropout(p=0.1

In [9]:
Criterion = nn.CrossEntropyLoss()

Optimizer = torch.optim.Adam(Model.parameters(), lr=0.0001)

In [10]:
train_loss = []
train_acc = []
test_loss = []
test_acc  = []

for epoch in range(200):
    
    train_loss_sum, test_loss_sum, train_num, test_num, train_i, test_i = 0.0, 0.0, 0, 0, 0, 0
    train_acc_sum,test_acc_sum = 0, 0
    TEST_acc_sum = 0
    
    for i_1, train_data in enumerate(train_loader):
        
        inputs, labels = train_data
        inputs = inputs.unsqueeze(1)
        inputs = inputs.type(torch.FloatTensor)
        inputs = inputs.to(device)
        labels = labels.long()
        labels = labels.squeeze(1)
        labels = labels.to(device)
        
        #print(inputs.shape)
        #print(labels.shape)
        
        Model.train()
        pre_labs = Model(inputs)

        Loss = Criterion(pre_labs, labels)

        Optimizer.zero_grad()
        Loss.backward()
        Optimizer.step()
        
        train_loss_sum += Loss.item()
        train_acc_sum += (pre_labs.argmax(dim=1) == labels).sum().item()
        train_num += labels.shape[0]
        train_i += 1

    for i_2, data in enumerate(test_loader):
        with torch.no_grad():

            test_data, test_lab = data  
            test_data = test_data.unsqueeze(1)
            test_data = test_data.type(torch.FloatTensor)
            test_data = test_data.to(device)
            test_lab = test_lab.long()
            test_lab = test_lab.squeeze(1)
            test_lab = test_lab.to(device)

            Model.eval()
            pre_test = Model(test_data)
            t_loss = Criterion(pre_test, test_lab)

            test_loss_sum += t_loss    
            test_acc_sum += (pre_test.argmax(dim=1) == test_lab).sum().item()
            test_num += test_lab.shape[0]                    
            test_i += 1
            
            #_, Pred = torch.max(pre_test.data, 1)
            #TEST_acc_sum += torch.sum(Pred == test_lab)
           
    if epoch >= 120:
        torch.save(Model, "model/4/CNN_%d.pkl"% epoch)
        print("-------保存第%d epoch的模型---------"% epoch)
       
    Train_Loss = train_loss_sum/train_i
    Test_Loss = test_loss_sum/test_i
    Train_ACC = train_acc_sum/train_num
    Test_ACC = test_acc_sum/test_num

    print('Epoch:%d, train_loss:%.5f, train_acc:%.5f, test_loss:%.5f, test_acc:%.5f' % 
          (epoch, Train_Loss, Train_ACC, Test_Loss, Test_ACC))
    print('-----------------------------------------------')

    train_loss.append(Train_Loss)            
    train_acc.append(Train_ACC) 
    test_loss.append(Test_Loss) 
    test_acc.append(Test_ACC)       

print("Finished Training")

Epoch:0, train_loss:1.94776, train_acc:0.14592, test_loss:1.94103, test_acc:0.14339
-----------------------------------------------
Epoch:1, train_loss:1.92504, train_acc:0.18211, test_loss:1.86377, test_acc:0.23410
-----------------------------------------------
Epoch:2, train_loss:1.80445, train_acc:0.27962, test_loss:1.72228, test_acc:0.32757
-----------------------------------------------
Epoch:3, train_loss:1.69883, train_acc:0.33342, test_loss:1.69518, test_acc:0.31610
-----------------------------------------------
Epoch:4, train_loss:1.63090, train_acc:0.36698, test_loss:1.59889, test_acc:0.36200
-----------------------------------------------
Epoch:5, train_loss:1.55792, train_acc:0.39303, test_loss:1.49745, test_acc:0.40155
-----------------------------------------------
Epoch:6, train_loss:1.49827, train_acc:0.41753, test_loss:1.44530, test_acc:0.41247
-----------------------------------------------
Epoch:7, train_loss:1.43818, train_acc:0.43904, test_loss:1.37542, test_acc:

Epoch:62, train_loss:0.45419, train_acc:0.83265, test_loss:0.47939, test_acc:0.82052
-----------------------------------------------
Epoch:63, train_loss:0.45581, train_acc:0.83179, test_loss:0.48187, test_acc:0.82342
-----------------------------------------------
Epoch:64, train_loss:0.44816, train_acc:0.83211, test_loss:0.46507, test_acc:0.82951
-----------------------------------------------
Epoch:65, train_loss:0.44394, train_acc:0.83496, test_loss:0.45114, test_acc:0.83449
-----------------------------------------------
Epoch:66, train_loss:0.44106, train_acc:0.83727, test_loss:0.50876, test_acc:0.80960
-----------------------------------------------
Epoch:67, train_loss:0.43704, train_acc:0.83632, test_loss:0.47560, test_acc:0.82163
-----------------------------------------------
Epoch:68, train_loss:0.41992, train_acc:0.84484, test_loss:0.43443, test_acc:0.83905
-----------------------------------------------
Epoch:69, train_loss:0.42535, train_acc:0.84289, test_loss:0.44029, t

-------保存第123 epoch的模型---------
Epoch:123, train_loss:0.24235, train_acc:0.91150, test_loss:0.35896, test_acc:0.87735
-----------------------------------------------
-------保存第124 epoch的模型---------
Epoch:124, train_loss:0.24036, train_acc:0.91209, test_loss:0.37752, test_acc:0.86809
-----------------------------------------------
-------保存第125 epoch的模型---------
Epoch:125, train_loss:0.23541, train_acc:0.91205, test_loss:0.36555, test_acc:0.87431
-----------------------------------------------
-------保存第126 epoch的模型---------
Epoch:126, train_loss:0.23116, train_acc:0.91418, test_loss:0.38695, test_acc:0.86988
-----------------------------------------------
-------保存第127 epoch的模型---------
Epoch:127, train_loss:0.23774, train_acc:0.91123, test_loss:0.41272, test_acc:0.85882
-----------------------------------------------
-------保存第128 epoch的模型---------
Epoch:128, train_loss:0.23704, train_acc:0.91282, test_loss:0.36313, test_acc:0.87334
-----------------------------------------------
----

-------保存第173 epoch的模型---------
Epoch:173, train_loss:0.16429, train_acc:0.94085, test_loss:0.37364, test_acc:0.87680
-----------------------------------------------
-------保存第174 epoch的模型---------
Epoch:174, train_loss:0.16628, train_acc:0.94008, test_loss:0.38383, test_acc:0.87721
-----------------------------------------------
-------保存第175 epoch的模型---------
Epoch:175, train_loss:0.16876, train_acc:0.93791, test_loss:0.36478, test_acc:0.88399
-----------------------------------------------
-------保存第176 epoch的模型---------
Epoch:176, train_loss:0.16724, train_acc:0.93845, test_loss:0.34038, test_acc:0.88827
-----------------------------------------------
-------保存第177 epoch的模型---------
Epoch:177, train_loss:0.16594, train_acc:0.93913, test_loss:0.36699, test_acc:0.88164
-----------------------------------------------
-------保存第178 epoch的模型---------
Epoch:178, train_loss:0.16297, train_acc:0.93881, test_loss:0.34982, test_acc:0.88537
-----------------------------------------------
----